# 前情
食物品种共有11类：Bread, Dairy product, Dessert, Egg, Fried food, Meat, Noodles/Pasta, Rice, Seafood, Soup, and Vegetable/Fruit.

Training set: 9866张

Validation set: 3430张

Testing set: 3347张

In [1]:
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms #torchvision是pytorch的一个图形库
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import time

# Read image 
利用 OpenCV (cv2) 讀入照片，並将照片转换成指定的分辨率，转化成数组存放在 numpy array 中

In [2]:
def readfile(path, label):# label 是0 或 1，1代表需要回傳 y 值
    image_dir = sorted(os.listdir(path))
    # uint8是专门用于存储各种图像的，范围是从0–255
    x = np.zeros((len(image_dir), 128, 128, 3), dtype=np.uint8) #初始化，
    y = np.zeros((len(image_dir)), dtype=np.uint8) #初始化
    # 给文件夹里的图片一个编号，并将编号和图片组合成一个表
    for i, file in enumerate(image_dir):
        # cv.imread(filename[, flags]) 得到图像的通道和色彩信息，使用默认参数: 8位深度，3通道
        # 结果是一个 高度X宽度 的矩阵，里面的元素是一个3维行向量，表示RGB的值
        img = cv2.imread(os.path.join(path, file)) #将文件夹位置和图片名拼接起来，得到图片位置
        x[i, :, :] = cv2.resize(img,(128, 128)) # 图片分辨率缩放成 128 * 128，然后放入矩阵
        if label:
            # 训练集图像命名方式为 [类别]_[第几张图片].jpg
            y[i] = int(file.split("_")[0]) #图片名分成2个部分，取前面的一个，得到类别作为 y 值
    if label:
      return x, y
    else:
      return x

In [3]:
# 分別將 training set、validation set、testing set 用 readfile 函式讀進來
workspace_dir = 'hw3_data/'
start_time = time.time()
print("Reading data")
train_x, train_y = readfile(os.path.join(workspace_dir, "training"), True)
print("Size of training data = {}".format(len(train_x)))
val_x, val_y = readfile(os.path.join(workspace_dir, "validation"), True)
print("Size of validation data = {}".format(len(val_x)))
test_x = readfile(os.path.join(workspace_dir, "testing"), False)
print("Size of Testing data = {}".format(len(test_x)))
end_time = time.time()
print("用时：", end_time - start_time)

Reading data
Size of training data = 9866
Size of validation data = 3430
Size of Testing data = 3347
用时： 152.72399735450745


# 预处理数据
在 PyTorch 中，我們可以利用 torch.utils.data 的 Dataset 及 DataLoader 來"包裝" data，使後續的 training 及 testing 更為方便。

Dataset 需要 overload 兩個函數：__len__ 及 __getitem__

__len__ 必須要回傳 dataset 的大小，而 __getitem__ 則定義了當程式利用 [ ] 取值時，dataset 應該要怎麼回傳資料。

實際上我們並不會直接使用到這兩個函數，但是使用 DataLoader 在 enumerate Dataset 時會使用到，沒有實做的話會在程式運行階段出現 error。__ getitem __()和__len__() 两个函数必须重写

In [4]:
# 来自torchvision.transforms
# 训练数据做数据增强 (data augmentation)
train_transform = transforms.Compose([ # 处理图片, 用Compose把多个处理步骤整合到一起
    transforms.ToPILImage(), #把数据转换为tensfroms格式
    transforms.RandomHorizontalFlip(), # 隨機將圖片左右镜像
    transforms.RandomRotation(15), # 隨機旋轉圖片
    transforms.ToTensor(), # 將圖片轉成 Tensor，並把數值 normalize 到 [0,1] ,这个格式可以直接输入进神经网络了
])

# 测试数据不需做 数据增强 (data augmentation)
test_transform = transforms.Compose([
    transforms.ToPILImage(),                                    
    transforms.ToTensor(),
])

# 定义ImgDataset类，继承torch.utils.data.Dataset，实现数据读取方式
# torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False,...) 
class ImgDataset(Dataset):
    def __init__(self, x, y=None, transform=None):
        self.x = x
        self.y = y
        if y is not None: #将 y 变成一个长向量 64-bit integer (signed)
            self.y = torch.LongTensor(y)
        self.transform = transform
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index): #取一张图片和对应的分类 y 值
        X = self.x[index]
        if self.transform is not None:
            X = self.transform(X)
        if self.y is not None: # 判断是否需要 y 值
            Y = self.y[index]
            return X, Y
        else:
            return X

调用上面的定义

In [5]:
batch_size = 128
train_set = ImgDataset(train_x, train_y, train_transform) #训练集
val_set = ImgDataset(val_x, val_y, test_transform) #验证集

# 使用torch.utils.data.DataLoader(), 实现数据的批量读取
#dataset：加载的数据集(Dataset对象); batch_size：batch size; shuffle:：是否将数据打乱
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

# Model

**```torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)```**

每个“filter”实际上是“卷积核”的一个集合, 每个通道都对应一个卷积核, 且这个卷积核是独一无二的, 每一个通道进行卷积运算,最后相加,形成一个filter 的单通道输出
- **in_channels** 每个卷积层中卷积核的数量
- **out_channels** 层中filter的个数，每个filter输出一个矩阵
- **kernel_size** 卷积核的大小，卷积核是一个二维方阵，只需要指定一维
- **步长(Stride)** 滑动filter时，先从输入的左上角开始，每次往左滑动一列或者往下滑动一行逐一计算输出，将每次滑动的行数和列数称为Stride
- **填充（Padding）** 用filter扫描图片时，中心的像素会被扫描多次，而图片边缘的像素只会被扫描一次，导致丢失了图片边界处的信息。有时我们还希望输入和输出的大小应该保持一致。为解决这个问题，可以在矩阵的边界上填充一些值，以增加矩阵的大小 
<img src="hw3_data/pading.gif" width="20%"> 

---

**```torch.nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True)```**

批标准化(Batch Normalization)操作:
- 先对输入进行归一化，$\bar X$为均值，Var(x)为方差, eps 防止分母为 0
- 然后对归一化的结果进行缩放和平移，设置affine=True，即意味着缩放(γ)和平移(β)将被使用
- momentum：动态均值和动态方差所使用的动量 $ x_{new} = (1 - momentum)\times x + momentum \times x_t $ 其中：x是估计的数据 $x_t$是新的观察到的数据
- 默认情况下 $ \gamma =1, bias = 0, momentum = 0.1, eps = 10^{-5} $
- num_features: 输入的层数
$$ y = {{x-\bar X} \over {\sqrt {var(X)+eps}}} \times \gamma + bias $$

---

**```torch.nn.MaxPool2d(kernel_size, stride, padding)```**

- **kernel_size** max pooling的窗口大小
- **stride** max pooling的窗口移动的步长，默认值是kernel_size
- **padding** 输入的每一条边补充0的个数

In [6]:
# import torch.nn as nn
class Classifier(nn.Module):
    def __init__(self): # input 維度 [3, 128, 128]
        super(Classifier, self).__init__() # super() 初始化 Classifier 的父类 nn.Module 的属性,避免直接使用父类的名字
        self.cnn = nn.Sequential(
            # 这层有64个filter
            nn.Conv2d(3, 64, 3, 1, 1), # [in_channels = 3, out_channels = 64, kernel_size = 3, stride = 1, padding = 1] 输出[64, 128, 128]
            nn.BatchNorm2d(64), #64 层输入
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # 一张图片用[64, 64, 64]矩阵表示
            
            # 这层有128个filter
            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            # 这层有256个filter
            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            # 这层有512个filter
            nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]
            
            # 这层有512个filter
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
        )
        
        self.fc = nn.Sequential(
            # 一个有序的容器，神经网络模块将按照在传入构造器的顺序依次被添加到计算图中执行
            # 以神经网络模块为元素的有序字典也可以作为传入参数。
            nn.Linear(512*4*4, 1024), #第1层，有 1024 个 neuron，输入 512*4*4 个feature, 输出 1024 维结果
            nn.ReLU(),
            nn.Linear(1024, 512), # 第2层，有 512 个 neuron，输入 1024 个feature, 输出 512 维结果
            nn.ReLU(),
            nn.Linear(512, 11) # 第3层，有 11 个 neuron，输入 512 个feature, 输出 11 维结果，最后一层不需要添加激活函数
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1) # 摊平成1维
        return self.fc(out)

助教写的model，叠很多层时用这种写法
``` python
class Classifier(nn.Module):
  def __init__(self):
    super(Classifier, self).__init__()

    def building_block(indim, outdim):
      return [ nn.Conv2d(indim, outdim, 3, 1, 1),
            nn.BatchNorm2d(outdim),
            nn.ReLU(),
      ]
    def stack_blocks(indim, outdim, block_num):
      layers = building_block(indim, outdim)
      for i in range(block_num - 1):
        layers += building_block(outdim, outdim)
      layers.append(nn.MaxPool2d(2, 2, 0))
      return layers

    cnn_list = []
    cnn_list += stack_blocks(3, 128, 3)
    cnn_list += stack_blocks(128, 128, 3)
    cnn_list += stack_blocks(128, 256, 3)
    cnn_list += stack_blocks(256, 512, 1)
    cnn_list += stack_blocks(512, 512, 1)
    self.cnn = nn.Sequential( * cnn_list)

    dnn_list = [
      nn.Linear(512 * 4 * 4, 1024),
      nn.ReLU(),
      nn.Dropout(p = 0.3),
      nn.Linear(1024, 11),
    ]
    self.fc = nn.Sequential( * dnn_list)

  def forward(self, x):
    out = self.cnn(x)
    out = out.reshape(out.size()[0], -1)
    return self.fc(out)
```

# Training
使用 training set 訓練，並使用 validation set 尋找好的參數

In [7]:
# 初始化
model = Classifier().cuda() # model是Classifier()类
loss = nn.CrossEntropyLoss() # 因為是 classification task，所以 loss 使用 CrossEntropyLoss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # optimizer 使用 Adam优化算法
num_epoch = 30

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    model.train() # 確保 model 是在 train model (開啟 Dropout 等...)
    for i, data in enumerate(train_loader): # 给train_loader的矩阵一个编号，并组合成一个表
        optimizer.zero_grad() # 用 optimizer 將 model 參數的 gradient 歸零
        train_pred = model(data[0].cuda()) # 利用 model 算出来的機率分佈 這邊實際上就是去呼叫 model 的 forward 函數
        batch_loss = loss(train_pred, data[1].cuda()) # 計算 loss （注意 prediction 跟 label 必須同時在 CPU 或是 GPU 上）
        batch_loss.backward() # 利用 back propagation 算出每個參數的 gradient
        optimizer.step() # 以 optimizer 用 gradient 更新參數值

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        train_loss += batch_loss.item()
    
    model.eval() # 不启用 BatchNormalization 和 Dropout 训练完train样本后，生成的模型model用来测试样本
    with torch.no_grad(): #torch.no_grad() 是一个上下文管理器，被该语句 wrap 起来的部分将不会track 梯度
        for i, data in enumerate(val_loader):
            val_pred = model(data[0].cuda())
            batch_loss = loss(val_pred, data[1].cuda())

            val_acc += np.sum(np.argmax(val_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
            val_loss += batch_loss.item()

        #將結果 print 出來
        print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
            (epoch + 1, num_epoch, time.time()-epoch_start_time, \
             train_acc/train_set.__len__(), train_loss/train_set.__len__(), val_acc/val_set.__len__(), val_loss/val_set.__len__()))

[001/030] 40.82 sec(s) Train Acc: 0.238699 Loss: 0.017742 | Val Acc: 0.285423 loss: 0.016074
[002/030] 39.53 sec(s) Train Acc: 0.332455 Loss: 0.014925 | Val Acc: 0.313994 loss: 0.015597
[003/030] 39.89 sec(s) Train Acc: 0.398034 Loss: 0.013621 | Val Acc: 0.384840 loss: 0.014021
[004/030] 39.66 sec(s) Train Acc: 0.440908 Loss: 0.012710 | Val Acc: 0.377843 loss: 0.014076
[005/030] 39.76 sec(s) Train Acc: 0.481553 Loss: 0.011789 | Val Acc: 0.482507 loss: 0.011911
[006/030] 39.80 sec(s) Train Acc: 0.511960 Loss: 0.011068 | Val Acc: 0.346939 loss: 0.016360
[007/030] 39.81 sec(s) Train Acc: 0.541962 Loss: 0.010514 | Val Acc: 0.413411 loss: 0.014510
[008/030] 39.85 sec(s) Train Acc: 0.563450 Loss: 0.009923 | Val Acc: 0.534694 loss: 0.010878
[009/030] 40.05 sec(s) Train Acc: 0.591831 Loss: 0.009240 | Val Acc: 0.385714 loss: 0.018962
[010/030] 39.95 sec(s) Train Acc: 0.613623 Loss: 0.008691 | Val Acc: 0.525073 loss: 0.011558
[011/030] 39.89 sec(s) Train Acc: 0.636327 Loss: 0.008249 | Val Acc: 0

得到好的參數後，我們使用 training set 和 validation set 共同訓練（資料量變多，模型效果較好）

In [9]:
train_val_x = np.concatenate((train_x, val_x), axis=0)
train_val_y = np.concatenate((train_y, val_y), axis=0)
train_val_set = ImgDataset(train_val_x, train_val_y, train_transform)
train_val_loader = DataLoader(train_val_set, batch_size=batch_size, shuffle=True)

In [10]:
# 和上面一样，省略验证部分
model_best = Classifier().cuda()
loss = nn.CrossEntropyLoss() # 因為是 classification task，所以 loss 使用 CrossEntropyLoss
optimizer = torch.optim.Adam(model_best.parameters(), lr=0.001) # optimizer 使用 Adam
num_epoch = 30

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0

    model_best.train()
    for i, data in enumerate(train_val_loader):
        optimizer.zero_grad()
        train_pred = model_best(data[0].cuda())
        batch_loss = loss(train_pred, data[1].cuda())
        batch_loss.backward()
        optimizer.step()

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        train_loss += batch_loss.item()

        #將結果 print 出來
    print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f' % \
      (epoch + 1, num_epoch, time.time()-epoch_start_time, \
      train_acc/train_val_set.__len__(), train_loss/train_val_set.__len__()))

[001/030] 47.10 sec(s) Train Acc: 0.240824 Loss: 0.017076
[002/030] 47.49 sec(s) Train Acc: 0.360860 Loss: 0.014254
[003/030] 47.57 sec(s) Train Acc: 0.438252 Loss: 0.012693
[004/030] 47.38 sec(s) Train Acc: 0.494585 Loss: 0.011465
[005/030] 47.53 sec(s) Train Acc: 0.531062 Loss: 0.010478
[006/030] 47.63 sec(s) Train Acc: 0.568893 Loss: 0.009651
[007/030] 47.79 sec(s) Train Acc: 0.604091 Loss: 0.008984
[008/030] 48.21 sec(s) Train Acc: 0.626053 Loss: 0.008417
[009/030] 48.74 sec(s) Train Acc: 0.647864 Loss: 0.007917
[010/030] 48.10 sec(s) Train Acc: 0.677647 Loss: 0.007304
[011/030] 49.27 sec(s) Train Acc: 0.697277 Loss: 0.006866
[012/030] 48.25 sec(s) Train Acc: 0.707882 Loss: 0.006542
[013/030] 48.82 sec(s) Train Acc: 0.719540 Loss: 0.006257
[014/030] 47.94 sec(s) Train Acc: 0.741652 Loss: 0.005791
[015/030] 47.96 sec(s) Train Acc: 0.756619 Loss: 0.005474
[016/030] 48.14 sec(s) Train Acc: 0.763237 Loss: 0.005330
[017/030] 48.12 sec(s) Train Acc: 0.784672 Loss: 0.004811
[018/030] 48.0

# Testing
利用剛剛 train 好的 model 進行 prediction

In [11]:
test_set = ImgDataset(test_x, transform=test_transform)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [12]:
model_best.eval()
prediction = []
with torch.no_grad():
    for i, data in enumerate(test_loader):
        test_pred = model_best(data.cuda())
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        for y in test_label:
            prediction.append(y)

In [19]:
name = ['面包', '奶', '甜品', '蛋', '油炸食品', '肉', '面条', '米饭', '海鲜', '汤', '果蔬']

In [20]:
#將結果寫入 csv 檔
with open("hw3_data/预测.csv", 'w') as f:
    f.write('Id,Category\n')
    for i, y in  enumerate(prediction):
        f.write('{},{},{}\n'.format(i, y, name[y]))

# 重命名测试集

In [34]:
# 获取该目录下所有文件，存入列表中
f = os.listdir("hw3_data/testing_prediction")
for i in range(len(f)):
    oldname = f[i]
    newname = oldname.split(".")[0] + '-'+ name[prediction[i]] + '.jpg'
    # 用os模块中的rename方法对文件改名
    os.rename("hw3_data/testing_prediction/" + oldname, "hw3_data/testing_prediction/" + newname)

0000.jpg ======> 0000-甜品.jpg
0001.jpg ======> 0001-果蔬.jpg
0002.jpg ======> 0002-海鲜.jpg
0003.jpg ======> 0003-汤.jpg
0004.jpg ======> 0004-米饭.jpg
0005.jpg ======> 0005-蛋.jpg
0006.jpg ======> 0006-汤.jpg
0007.jpg ======> 0007-果蔬.jpg
0008.jpg ======> 0008-蛋.jpg
0009.jpg ======> 0009-甜品.jpg
0010.jpg ======> 0010-油炸食品.jpg
0011.jpg ======> 0011-面包.jpg
0012.jpg ======> 0012-面条.jpg
0013.jpg ======> 0013-果蔬.jpg
0014.jpg ======> 0014-面包.jpg
0015.jpg ======> 0015-油炸食品.jpg
0016.jpg ======> 0016-汤.jpg
0017.jpg ======> 0017-油炸食品.jpg
0018.jpg ======> 0018-甜品.jpg
0019.jpg ======> 0019-汤.jpg
0020.jpg ======> 0020-蛋.jpg
0021.jpg ======> 0021-肉.jpg
0022.jpg ======> 0022-蛋.jpg
0023.jpg ======> 0023-蛋.jpg
0024.jpg ======> 0024-汤.jpg
0025.jpg ======> 0025-蛋.jpg
0026.jpg ======> 0026-果蔬.jpg
0027.jpg ======> 0027-油炸食品.jpg
0028.jpg ======> 0028-汤.jpg
0029.jpg ======> 0029-海鲜.jpg
0030.jpg ======> 0030-甜品.jpg
0031.jpg ======> 0031-汤.jpg
0032.jpg ======> 0032-甜品.jpg
0033.jpg ======> 0033-面包.jpg
0034.jpg ======> 003

0503.jpg ======> 0503-汤.jpg
0504.jpg ======> 0504-油炸食品.jpg
0505.jpg ======> 0505-面条.jpg
0506.jpg ======> 0506-肉.jpg
0507.jpg ======> 0507-海鲜.jpg
0508.jpg ======> 0508-果蔬.jpg
0509.jpg ======> 0509-果蔬.jpg
0510.jpg ======> 0510-甜品.jpg
0511.jpg ======> 0511-面条.jpg
0512.jpg ======> 0512-汤.jpg
0513.jpg ======> 0513-油炸食品.jpg
0514.jpg ======> 0514-油炸食品.jpg
0515.jpg ======> 0515-甜品.jpg
0516.jpg ======> 0516-汤.jpg
0517.jpg ======> 0517-汤.jpg
0518.jpg ======> 0518-肉.jpg
0519.jpg ======> 0519-面包.jpg
0520.jpg ======> 0520-面包.jpg
0521.jpg ======> 0521-蛋.jpg
0522.jpg ======> 0522-米饭.jpg
0523.jpg ======> 0523-汤.jpg
0524.jpg ======> 0524-肉.jpg
0525.jpg ======> 0525-甜品.jpg
0526.jpg ======> 0526-汤.jpg
0527.jpg ======> 0527-汤.jpg
0528.jpg ======> 0528-面条.jpg
0529.jpg ======> 0529-面包.jpg
0530.jpg ======> 0530-油炸食品.jpg
0531.jpg ======> 0531-汤.jpg
0532.jpg ======> 0532-面包.jpg
0533.jpg ======> 0533-汤.jpg
0534.jpg ======> 0534-汤.jpg
0535.jpg ======> 0535-果蔬.jpg
0536.jpg ======> 0536-奶.jpg
0537.jpg ======> 0537

1038.jpg ======> 1038-肉.jpg
1039.jpg ======> 1039-海鲜.jpg
1040.jpg ======> 1040-汤.jpg
1041.jpg ======> 1041-海鲜.jpg
1042.jpg ======> 1042-汤.jpg
1043.jpg ======> 1043-油炸食品.jpg
1044.jpg ======> 1044-甜品.jpg
1045.jpg ======> 1045-汤.jpg
1046.jpg ======> 1046-汤.jpg
1047.jpg ======> 1047-肉.jpg
1048.jpg ======> 1048-油炸食品.jpg
1049.jpg ======> 1049-汤.jpg
1050.jpg ======> 1050-蛋.jpg
1051.jpg ======> 1051-蛋.jpg
1052.jpg ======> 1052-汤.jpg
1053.jpg ======> 1053-肉.jpg
1054.jpg ======> 1054-海鲜.jpg
1055.jpg ======> 1055-汤.jpg
1056.jpg ======> 1056-面包.jpg
1057.jpg ======> 1057-肉.jpg
1058.jpg ======> 1058-蛋.jpg
1059.jpg ======> 1059-汤.jpg
1060.jpg ======> 1060-果蔬.jpg
1061.jpg ======> 1061-果蔬.jpg
1062.jpg ======> 1062-肉.jpg
1063.jpg ======> 1063-肉.jpg
1064.jpg ======> 1064-汤.jpg
1065.jpg ======> 1065-肉.jpg
1066.jpg ======> 1066-甜品.jpg
1067.jpg ======> 1067-油炸食品.jpg
1068.jpg ======> 1068-油炸食品.jpg
1069.jpg ======> 1069-汤.jpg
1070.jpg ======> 1070-汤.jpg
1071.jpg ======> 1071-面条.jpg
1072.jpg ======> 1072-面包.jp

1395.jpg ======> 1395-甜品.jpg
1396.jpg ======> 1396-果蔬.jpg
1397.jpg ======> 1397-甜品.jpg
1398.jpg ======> 1398-海鲜.jpg
1399.jpg ======> 1399-汤.jpg
1400.jpg ======> 1400-海鲜.jpg
1401.jpg ======> 1401-汤.jpg
1402.jpg ======> 1402-汤.jpg
1403.jpg ======> 1403-汤.jpg
1404.jpg ======> 1404-汤.jpg
1405.jpg ======> 1405-油炸食品.jpg
1406.jpg ======> 1406-蛋.jpg
1407.jpg ======> 1407-油炸食品.jpg
1408.jpg ======> 1408-面包.jpg
1409.jpg ======> 1409-甜品.jpg
1410.jpg ======> 1410-面包.jpg
1411.jpg ======> 1411-油炸食品.jpg
1412.jpg ======> 1412-汤.jpg
1413.jpg ======> 1413-海鲜.jpg
1414.jpg ======> 1414-海鲜.jpg
1415.jpg ======> 1415-肉.jpg
1416.jpg ======> 1416-海鲜.jpg
1417.jpg ======> 1417-面包.jpg
1418.jpg ======> 1418-蛋.jpg
1419.jpg ======> 1419-面包.jpg
1420.jpg ======> 1420-肉.jpg
1421.jpg ======> 1421-甜品.jpg
1422.jpg ======> 1422-蛋.jpg
1423.jpg ======> 1423-甜品.jpg
1424.jpg ======> 1424-果蔬.jpg
1425.jpg ======> 1425-甜品.jpg
1426.jpg ======> 1426-油炸食品.jpg
1427.jpg ======> 1427-面包.jpg
1428.jpg ======> 1428-甜品.jpg
1429.jpg ======> 

1844.jpg ======> 1844-肉.jpg
1845.jpg ======> 1845-肉.jpg
1846.jpg ======> 1846-肉.jpg
1847.jpg ======> 1847-甜品.jpg
1848.jpg ======> 1848-面包.jpg
1849.jpg ======> 1849-海鲜.jpg
1850.jpg ======> 1850-甜品.jpg
1851.jpg ======> 1851-肉.jpg
1852.jpg ======> 1852-面包.jpg
1853.jpg ======> 1853-油炸食品.jpg
1854.jpg ======> 1854-米饭.jpg
1855.jpg ======> 1855-奶.jpg
1856.jpg ======> 1856-面条.jpg
1857.jpg ======> 1857-汤.jpg
1858.jpg ======> 1858-奶.jpg
1859.jpg ======> 1859-面条.jpg
1860.jpg ======> 1860-海鲜.jpg
1861.jpg ======> 1861-汤.jpg
1862.jpg ======> 1862-面包.jpg
1863.jpg ======> 1863-蛋.jpg
1864.jpg ======> 1864-海鲜.jpg
1865.jpg ======> 1865-面包.jpg
1866.jpg ======> 1866-面包.jpg
1867.jpg ======> 1867-面包.jpg
1868.jpg ======> 1868-奶.jpg
1869.jpg ======> 1869-面包.jpg
1870.jpg ======> 1870-奶.jpg
1871.jpg ======> 1871-肉.jpg
1872.jpg ======> 1872-面包.jpg
1873.jpg ======> 1873-甜品.jpg
1874.jpg ======> 1874-面包.jpg
1875.jpg ======> 1875-汤.jpg
1876.jpg ======> 1876-海鲜.jpg
1877.jpg ======> 1877-蛋.jpg
1878.jpg ======> 1878-面包.j

2358.jpg ======> 2358-甜品.jpg
2359.jpg ======> 2359-甜品.jpg
2360.jpg ======> 2360-油炸食品.jpg
2361.jpg ======> 2361-米饭.jpg
2362.jpg ======> 2362-蛋.jpg
2363.jpg ======> 2363-汤.jpg
2364.jpg ======> 2364-蛋.jpg
2365.jpg ======> 2365-面包.jpg
2366.jpg ======> 2366-面包.jpg
2367.jpg ======> 2367-油炸食品.jpg
2368.jpg ======> 2368-面包.jpg
2369.jpg ======> 2369-蛋.jpg
2370.jpg ======> 2370-甜品.jpg
2371.jpg ======> 2371-汤.jpg
2372.jpg ======> 2372-油炸食品.jpg
2373.jpg ======> 2373-面包.jpg
2374.jpg ======> 2374-肉.jpg
2375.jpg ======> 2375-甜品.jpg
2376.jpg ======> 2376-面包.jpg
2377.jpg ======> 2377-海鲜.jpg
2378.jpg ======> 2378-蛋.jpg
2379.jpg ======> 2379-肉.jpg
2380.jpg ======> 2380-汤.jpg
2381.jpg ======> 2381-肉.jpg
2382.jpg ======> 2382-汤.jpg
2383.jpg ======> 2383-油炸食品.jpg
2384.jpg ======> 2384-果蔬.jpg
2385.jpg ======> 2385-油炸食品.jpg
2386.jpg ======> 2386-面包.jpg
2387.jpg ======> 2387-油炸食品.jpg
2388.jpg ======> 2388-蛋.jpg
2389.jpg ======> 2389-面包.jpg
2390.jpg ======> 2390-面包.jpg
2391.jpg ======> 2391-海鲜.jpg
2392.jpg =====

2880.jpg ======> 2880-甜品.jpg
2881.jpg ======> 2881-蛋.jpg
2882.jpg ======> 2882-面条.jpg
2883.jpg ======> 2883-海鲜.jpg
2884.jpg ======> 2884-油炸食品.jpg
2885.jpg ======> 2885-汤.jpg
2886.jpg ======> 2886-蛋.jpg
2887.jpg ======> 2887-甜品.jpg
2888.jpg ======> 2888-果蔬.jpg
2889.jpg ======> 2889-蛋.jpg
2890.jpg ======> 2890-汤.jpg
2891.jpg ======> 2891-海鲜.jpg
2892.jpg ======> 2892-汤.jpg
2893.jpg ======> 2893-面包.jpg
2894.jpg ======> 2894-甜品.jpg
2895.jpg ======> 2895-甜品.jpg
2896.jpg ======> 2896-汤.jpg
2897.jpg ======> 2897-海鲜.jpg
2898.jpg ======> 2898-汤.jpg
2899.jpg ======> 2899-奶.jpg
2900.jpg ======> 2900-汤.jpg
2901.jpg ======> 2901-蛋.jpg
2902.jpg ======> 2902-海鲜.jpg
2903.jpg ======> 2903-蛋.jpg
2904.jpg ======> 2904-蛋.jpg
2905.jpg ======> 2905-汤.jpg
2906.jpg ======> 2906-油炸食品.jpg
2907.jpg ======> 2907-甜品.jpg
2908.jpg ======> 2908-面包.jpg
2909.jpg ======> 2909-面包.jpg
2910.jpg ======> 2910-汤.jpg
2911.jpg ======> 2911-肉.jpg
2912.jpg ======> 2912-油炸食品.jpg
2913.jpg ======> 2913-肉.jpg
2914.jpg ======> 2914-汤.j